<a href="https://colab.research.google.com/github/CmilAmaya/intelligent_agents/blob/main/MCAR_practica3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3.5
### [Introducción a los sistemas inteligentes](https://fagonzalezo.github.io/)
___________

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/pHasW3C30nxzXzjLQl1k) antes del final de la clase. El archivo debe nombrarse como  `isi-practica3.5-unalusername.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad.

___________

## Cuadrado de Rubik (o Cubo de Rubik 2D)

El *Cuadrado de Rubik* es un rompecabezas inspirado en el famoso cubo de Rubik. Consiste de un arreglo de 16 fichas organizadas en una matriz de $4\times 4$ como se ilustra en la siguiente figura:

<img src="https://raw.githubusercontent.com/fagonzalezo/iis-2018-2/master/rubik2D.png"
alt="Cuadrado de Rubik " width="240" height="180" border="10" />

Los colores son ilustrativos, lo importante es el número en cada una de las fichas. Se pueden hacer 10 movimientos diferentes correspondientes a rotar las 4 fichas alrededor de cada uno de los puntos A, B, C, D y E en el sentido de las manecillas del reloj o en el sentido opuesto.

Su objetivo es modelar el *Cuadrado de Rubik* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


### 1. Cree una clase para modelar el problema del Cuadrado de Rubik

Un Cuadrado de Rubik debe representarse como una lista con valores enteros que representan cada una de las fichas.

Por ejemplo un Cuadrado de Rubik resuelto debe verse así:

```python
[ 1,  2,  3,  4,
  5,  6,  7,  8,
  9, 10, 11, 12,
 13, 14, 15, 16]
```

#### Definición de acciones

La siguiente lista define las posibles acciones que se pueden ejecutar:

In [5]:
'''
These values MUST not be changed.
They represent the movements of the Rubik's Square.
'''
ACTIONS = ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

Cada acción indica la posición y el sentido. Por ejemplo, `'C-'` rota la posición C en el sentido opuesto de las manecillas del reloj. Si aplicamos esta acción al estado solución se obtiene el estado:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 10, 12,
 13, 14, 15, 16]
```

Si sobre este estado, aplicamos la acción `'E+'` obtenemos:

```python
[ 1,  2,  3,  4,
  5,  7, 11,  8,
  9,  6, 15, 10,
 13, 14, 16, 12]
```

#### Clase Rubik2D_problem

In [18]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)


class Rubik2d_problem(Problem):

    def __init__(self, initial):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15]
        [1, 9, 4, 8, 5, 6, 3, 2, 15, 10, 11, 12, 13, 14, 7, 16]
        [2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12]
        '''
        self.expanded = 0
        self.goal = tuple([i for i in range(1, 17)])  # Goal state is numbers 1-16 in order
        super().__init__(initial=tuple(initial), goal=self.goal)

    def actions(self, state):
        """Return a list of actions that can be executed in the given
        state."""
        return ACTIONS  # Using the predefined actions list ["A+", "A-", "B+", "B-", "C+", "C-", "D+", "D-", "E+", "E-"]

    def result(self, state, action):
        """
        Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state).
        """
        new_state = list(state)  # Create a copy to avoid modifying the original state

        # Define which positions are affected by each rotation point
        rotations = {
            'A': [0, 1, 5, 4],    # Top-left rotation point
            'B': [2, 3, 7, 6],
            'C': [5, 6, 10, 9],
            'D': [8, 9, 13, 12],
            'E': [10, 11, 15, 14],

        }

        point = action[0]  # Get the rotation point (A, B, C, D, or E)
        direction = action[1]  # Get the direction (+ or -)
        affected_positions = rotations[point]

        if direction == '+':  # Clockwise rotation
            values = [state[i] for i in affected_positions]
            values = [values[-1]] + values[:-1]  # Rotate right
        else:  # Counter-clockwise rotation
            values = [state[i] for i in affected_positions]
            values = values[1:] + [values[0]]  # Rotate left

        for pos, val in zip(affected_positions, values):
            new_state[pos] = val

        return tuple(new_state)  # Return the new state as a tuple

    def is_goal(self, state):
        '''
        Define when a given state is a goal state
        '''
        return state == self.goal

    def action_cost(self, s, a, s1):
        """
        Return the cost of a solution path that arrives at s1 from
        state s via action a.
        """
        return 1  # Uniform cost for all actions

    def h1(self, node):
      """The misplaced tiles heuristic."""
      return hamming_distance(node.state, self.goal)

    def h2(self, node):
      """The Manhattan heuristic."""
      X = (0, 1, 2, 3,
           0, 1, 2, 3,
           0, 1, 2, 3,
           0, 1, 2, 3)

      Y = (0, 0, 0, 0,
           1, 1, 1, 1,
           2, 2, 2, 2,
           3, 3, 3, 3)


      return sum(abs(X[s-1] - X[g-1]) + abs(Y[s-1] - Y[g-1])
                  for (s, g) in zip(node.state, self.goal))

    def h(self, node): return h2(self, node)

def hamming_distance(A, B):
  "Number of positions where vectors A and B are different."
  return sum(a != b for a, b in zip(A, B))

### 2. Evalue su código con diferentes estrategias de búsqueda



Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice las implementaciones de búsqueda en amplitud y búsqueda en profundidad iterativa.

Evaluelo para ver cuál es la máxima profundidad que se puede alcanzar en un tiempo razonable con cada estrategia de búsqueda. Reporte los resultados.

In [17]:
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

failure = object()
cutoff  = object()

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    N = 0
    while frontier:
        node = frontier.pop()

        if problem.is_goal(node.state):
            return node,N

        elif len(node) >= limit:
            result = cutoff

        elif not is_cycle(node):
            N+=1
            for child in expand(problem, node):
                frontier.append(child)
    return result


def g(n): return n.path_cost


def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    N = 0
    while frontier:
        node = frontier.pop()
        N+=1
        if problem.is_goal(node.state):

            return node, N
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))

def board16(board, fmt=(4 * '{} {} {} {}\n')):
    "A string representing an 16-puzzle board"
    return fmt.format(*board)

In [13]:
def bfs(problem):
    """
    Perform a breadth-first search on the problem.
    Return the list of actions to reach the goal state.
    """
    result = breadth_first_search(problem)
    return path_actions(result)

def iterativeDeepeningSearch(problem):
    "Do depth-limited search with increasing depth limits."
    result = iterative_deepening_search(problem)
    return path_actions(result[0])


# Creates a problem instance to simulate some moves
problem = Rubik2d_problem(list(range(1, 17)))
state = problem.initial
print(state)
state = problem.result(state, "A+")
print(state)
state = problem.result(state, "B-")

# Now you can test the search algorithms
problem = Rubik2d_problem(state)
actions = bfs(problem)
print(actions)
actions = iterativeDeepeningSearch(problem)
print(actions)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
(5, 1, 3, 4, 6, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
['A-', 'B+']
['B+', 'A-']


Try the following problem. What happens?

In [14]:
problem = Rubik2d_problem( [1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15])

state = problem.initial

actions = iterative_deepening_search(problem)
print(actions)


(<(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)>, 127)


## 3. Implemente diferente heurísticas para el problema

Consulte el código en el repositorio de AIMA (https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) y utilice la implementación de A*.

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de la 3ra edición de [Russell10].

In [15]:
from types import resolve_bases
def nullHeuristic(state):
    return 0

def aStarSearch(problem, h=nullHeuristic):
    """
    Perform an A*t search on the problem.
    Returns the list of actions to reach the goal state.
    """
    result = astar_search(problem, h)
    return path_actions(result[0])

def myHeuristic(state):
    ### your code here ###
    return 0



In [16]:
problem = Rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print(iterativeDeepeningSearch(problem))
print(aStarSearch(problem, myHeuristic))

['E-', 'D-', 'D-', 'B+', 'A+', 'C+']
['D+', 'D+', 'B+', 'E-', 'A+', 'C+']


In [24]:
problem = Rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])

print("ITERATIVE DEEPENING SEARCH")
result_ids = iterative_deepening_search(problem)
N_ids = result_ids[1]
depth = len(path_states(result_ids[0])) - 1 # Calculate depth from the path length
print("Nodos expandidos", N_ids)
print("IDN DEPTH", depth)
print("Branching factor", N_ids**(1/depth))
print()

print("A* WITH H1 ")
result_h1 = astar_search(problem, problem.h1)
N_h1 = result_h1[1]
depth = len(path_states(result_h1[0])) - 1 # Calculate depth from the path length
print("Nodos expandidos", N_h1)
print("A* (h1) DEPTH", depth)
print("Branching factor", N_h1**(1/depth))
print()

print("A* WITH H2 ")
result_h2 = astar_search(problem, problem.h2)
N_h2 = result_h2[1]
depth = len(path_states(result_h2[0])) - 1
print("Nodos expandidos", N_h2)
print("A* (h2) DEPTH", depth)
print("Branching factor", N_h2**(1/depth))

ITERATIVE DEEPENING SEARCH
Nodos expandidos 1048
IDN DEPTH 6
Branching factor 3.18708427191146

A* WITH H1 
Nodos expandidos 14
A* (h1) DEPTH 6
Branching factor 1.5524632891554087

A* WITH H2 
Nodos expandidos 7
A* (h2) DEPTH 6
Branching factor 1.3830875542684884
